In [ ]:
!nvidia-smi

Wed Mar 31 16:19:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# INSTALL

In [ ]:
!ln -s '/content/drive/MyDrive/THESIS/AI_CITY_2021/SOURCE' '/content'
!ln -s '/content/drive/MyDrive/THESIS/AI_CITY_2021/DATA' '/content'

In [ ]:
# %%capture 
!git clone https://github.com/lukemelas/EfficientNet-PyTorch.git
%cd /content/EfficientNet-PyTorch
!pip install -e .

Cloning into 'EfficientNet-PyTorch'...
remote: Enumerating objects: 584, done.
remote: Total 584 (delta 0), reused 0 (delta 0), pack-reused 584
Receiving objects: 100% (584/584), 1.11 MiB | 24.14 MiB/s, done.
Resolving deltas: 100% (308/308), done.
/content/EfficientNet-PyTorch
Obtaining file:///content/EfficientNet-PyTorch
  Running setup.py develop for efficientnet-pytorch


# DEPENDENCIES

In [ ]:
import os, json, cv2
import os.path as osp 
import pandas as pd 
from tqdm.notebook import tqdm
from glob import glob
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder

import pandas as pd 
import cv2, json, os, shutil
import numpy as np 
import matplotlib.pyplot as plt
import os.path as osp
from tqdm.notebook import tqdm
from glob import glob
from google.colab.patches import cv2_imshow

from torch.utils.data import Subset, DataLoader, Dataset

from efficientnet_pytorch import EfficientNet

import pickle

# CONFIGURATION

In [ ]:
TRACKS_DIR = '/content/SOURCE/vehicle_tracking/data/Centernet2_train_veh_boxes.json'
ROOT_DIR = '/content/DATA/data_track_5/AIC21_Track5_NL_Retrieval'

In [ ]:
CONFIG_COLOR = {
    'MODEL': 'efficientnet-b5',
    'SAVE_DIR': '/content/SOURCE/extract_object/results/color_classification_fraction',
    'SAVE_FEATURE_DIR': '/content/SOURCE/nanonets_object_tracking/feature/color',
    'EXP_ID': 'Mar26_b5',
    'NUM_CLASSES': 8,
    'image_size': (224,224),
    "imagenet_mean":[0.485, 0.456, 0.406],
    "imagenet_std":[0.229, 0.224, 0.225],
    'score_thres': 0.5,
    'seed': 88,
}

CONFIG_COLOR['SAVE_DIR'] = osp.join(CONFIG_COLOR['SAVE_DIR'], CONFIG_COLOR['EXP_ID'])
os.makedirs(CONFIG_COLOR['SAVE_DIR'], exist_ok=True)

# MODEL

In [ ]:
class Classifier(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        backbone = EfficientNet.from_pretrained(cfg['MODEL'], include_top=False)
        # self.backbone = self.backbone.cuda()
        out_channel = backbone._conv_head.out_channels

        self.feature_extractor = nn.Sequential(
            backbone,
            nn.Flatten()
        )
        # avg_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(out_channel, cfg['NUM_CLASSES'])
        )
        # self.classifier = nn.Linear(out_channel, cfg['NUM_CLASSES'])
        
    
    def extract_feature(self, input):
        x = self.feature_extractor(input)
        # x = self.avg_pool(x)
        return x
        
    def forward(self, input):
        x = self.extract_feature(input)
        logits = self.classifier(x)
        # logits = self.skeleton(input)
        logits = torch.softmax(logits, dim=-1)
        return logits
        

## Sanity check

In [ ]:
model = Classifier(CONFIG_COLOR)
model = model.cuda()
a = torch.rand((1, 3, 224, 224)).cuda()
a_feats = model.extract_feature(a)
a_pred = model(a)
print(f'feat shape: {a_feats.shape}')
print(f'pred shape: {a_pred.shape}')
print(a_feats.dtype)
print(a_pred.dtype)
print(a_feats.get_device())
print(a_pred.get_device())

# DATASET

In [ ]:
track = json.load(open(TRACKS_DIR))

# CHECKPOINT

In [ ]:
model_vehicle = Classifier(CONFIG_COLOR)

ckpt_path = osp.join(CONFIG_COLOR['SAVE_DIR'], 'best_model.pt')
if osp.isfile(ckpt_path):
    model_vehicle.load_state_dict(torch.load(ckpt_path))
    print(f'Loaded checkpoint from {ckpt_path}')

model_vehicle = model_vehicle.cuda()
model_vehicle.eval()

# INFERENCE

In [ ]:
import PIL
val_transform = transforms.Compose([
    transforms.Resize(CONFIG_COLOR['image_size'], PIL.Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(CONFIG_COLOR['imagenet_mean'], CONFIG_COLOR['imagenet_std']),
])

def preprocess_input(img):
    img = img.convert('RGB')
    img = val_transform(img)
    return img

In [ ]:
for track_id in tqdm(track):
    ans = {}
    ans[track_id] = {}
    for val in tqdm(track[track_id]):
        for key in val:
            ans[track_id][key] = []
            new_path = os.path.join(ROOT_DIR, key)
            img = Image.open(new_path)
            for box in val[key]:
                x_0, y_0, x_1, y_1 = box
                x_0, y_0, x_1, y_1 = int(x_0), int(y_0), int(x_1), int(y_1)
                crop_img = img.crop((x_0, y_0, x_1, y_1))
                crop_img = preprocess_input(crop_img).cuda()
                crop_img = crop_img.unsqueeze(0)
                with torch.no_grad():
                    feature = model_vehicle.extract_feature(crop_img)
                ans[track_id][key].append(feature)
    filename = f'{track_id}.pickle'
    save_dir = os.path.join(CONFIG_COLOR['SAVE_FEATURE_DIR'], filename)
    with open(save_dir, 'wb') as handle:
        pickle.dump(ans, handle, protocol=pickle.HIGHEST_PROTOCOL)

KeyboardInterrupt: ignored